# 📈 SpectraMind V50 — Ensembles, MC Dropout & COREL Conformal (Notebook 08)

**Goal.** Add **epistemic uncertainty** and **coverage-guaranteed intervals** on top of per-bin σ by:
- Training **ensembles** (multi-seed or multi-arch)
- Enabling **MC Dropout** at inference
- Running **post-hoc calibration** (temperature scaling; optional per-wavelength)
- Applying **COREL conformal prediction** for graph-aware coverage

**Workflow**
1. Pre-flight & environment capture
2. Hydra overrides for ensembles, MC dropout, calibration, and COREL
3. Train K ensemble members (looped CLI calls)
4. Predict per member; aggregate mean/variance
5. Temperature scaling (post-hoc)
6. COREL conformal calibration & coverage evaluation
7. Diagnostics dashboard + artifact tree

> As with earlier notebooks, this is **CLI-first & Hydra-safe** with **DRY-RUN** fallbacks when `spectramind` is unavailable.


In [ ]:
# ░░ Pre-flight: env, run IDs, paths, CLI detection ░░
import os, sys, json, platform, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"uq_conformal_{RUN_TS}"
ROOT_OUT = "/mnt/data/uq_conformal"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
PRED_OUT = os.path.join(ARTIFACTS, "predictions")
DIAG_OUT = os.path.join(ARTIFACTS, "diagnostics")
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, PRED_OUT, DIAG_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd: str) -> bool:
    return shutil.which(cmd) is not None

CLI_PRESENT = which("spectramind")
DRY_RUN = not CLI_PRESENT

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cli_present": CLI_PRESENT,
    "dry_run": DRY_RUN,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "predictions": PRED_OUT, "diagnostics": DIAG_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== SpectraMind V50 — Notebook 08 ===")
print(json.dumps(env, indent=2))


## Compose Hydra overrides for Ensembles, MC Dropout, Calibration & COREL

**Toggles (adjust to your repo’s `configs/`):**
- `uq.ensemble.enable=true`, `uq.ensemble.size=5`
- `uq.mc_dropout.enable=true`, with `keep_prob` or `p`, and `mc_samples=30`
- `calibration.temperature.enable=true` (optionally per-wavelength)
- `conformal.corel.enable=true`, e.g., `coverage=0.9`, graph spec for AIRS bins

We keep epochs small for demo; run ablations later for full sweeps.


In [ ]:
import json, os

overrides = {
    # Base runtime
    "data": "ariel_nominal",
    "model": "v50",                 # swap with 'hf_vit' or others if desired
    "training.max_epochs": "6",
    "training.batch_size": "16",
    "training.seed": "1337",
    # Ensemble
    "uq.ensemble.enable": "true",
    "uq.ensemble.size": "5",
    # MC Dropout
    "uq.mc_dropout.enable": "true",
    "uq.mc_dropout.p": "0.1",
    "uq.mc_dropout.mc_samples": "30",
    # Temperature scaling
    "calibration.temperature.enable": "true",
    "calibration.temperature.per_wavelength": "false",
    # COREL conformal
    "conformal.corel.enable": "true",
    "conformal.corel.coverage": "0.90",
    "conformal.corel.graph": "airs_default",
    # Mixed precision (optional)
    "training.mixed_precision": "fp16",
}

cfg_file = os.path.join(CFG_OUT, "uq_corel_overrides.json")
with open(cfg_file, "w") as f:
    json.dump(overrides, f, indent=2)
print("Saved overrides ->", cfg_file)
print(json.dumps(overrides, indent=2))


## Helper: robust CLI runner (DRY-RUN when CLI not present)

In [ ]:
import shlex, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": DRY_RUN, "returncode": 0, "stdout": "", "stderr": ""}
    if DRY_RUN:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        placeholder = os.path.join(ARTIFACTS, "dry_run_placeholder.txt")
        with open(placeholder, "a") as f: f.write(msg)
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try:
        result["stdout"] = open(log_path, "r").read()
    except Exception:
        pass
    try:
        result["stderr"] = open(err_path, "r").read()
    except Exception:
        pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## Train **K** ensemble members

In [ ]:
K = int(overrides.get("uq.ensemble.size", "5"))
base_seed = int(overrides.get("training.seed", "1337"))
run_dirs = []

for k in range(K):
    seed = base_seed + k
    run_name = f"member_{k:02d}_seed_{seed}"
    out_dir = os.path.join(PRED_OUT, run_name)
    os.makedirs(out_dir, exist_ok=True)
    run_dirs.append(out_dir)

    cmd = ["spectramind", "train",
           "--config-name", "config_v50.yaml",
           "+outputs.root_dir=" + out_dir,
           f"+training.seed={seed}"]
    for key, val in overrides.items():
        cmd.append(f"+{key}={val}")
    cmd += ["+training.fast_mode=true"]
    res = run_cli(cmd, log_name=f"01_train_{run_name}")
    print(res["stdout"][:300])


## Predict for each member and **aggregate**

In [ ]:
pred_dirs = []
for out_dir in run_dirs:
    pred_dir = os.path.join(out_dir, "pred")
    os.makedirs(pred_dir, exist_ok=True)
    pred_dirs.append(pred_dir)
    cmd_pred = ["spectramind", "predict",
                "--config-name", "config_v50.yaml",
                "+load.from_checkpoint=true",
                "+outputs.root_dir=" + pred_dir]
    for key, val in overrides.items():
        cmd_pred.append(f"+{key}={val}")
    cmd_pred += ["+inference.save_mu_sigma=true"]
    res_p = run_cli(cmd_pred, log_name=f"02_predict_{os.path.basename(out_dir)}")
    print(res_p["stdout"][:300])

agg_dir = os.path.join(PRED_OUT, "ensemble_aggregate")
os.makedirs(agg_dir, exist_ok=True)
cmd_agg = ["spectramind", "diagnose", "ensemble-aggregate",
           "--inputs"] + pred_dirs + ["--out", agg_dir]
res_agg = run_cli(cmd_agg, log_name="03_ensemble_aggregate")
print(res_agg["stdout"][:300])


## Temperature scaling (post-hoc)

In [ ]:
cal_dir = os.path.join(DIAG_OUT, "temperature_scaling")
os.makedirs(cal_dir, exist_ok=True)
cmd_ts = ["spectramind", "calibrate", "temperature",
          "--pred", os.path.join(PRED_OUT, "ensemble_aggregate"),
          "--out", cal_dir,
          "--mode", "global"]
res_ts = run_cli(cmd_ts, log_name="04_temperature_scaling")
print(res_ts["stdout"][:300])


## COREL conformal calibration

In [ ]:
corel_dir = os.path.join(DIAG_OUT, "corel")
os.makedirs(corel_dir, exist_ok=True)
cmd_corel_fit = ["spectramind", "conformal", "corel-fit",
                 "--pred", os.path.join(PRED_OUT, "ensemble_aggregate"),
                 "--out", corel_dir,
                 "--coverage", overrides.get("conformal.corel.coverage", "0.90"),
                 "--graph", overrides.get("conformal.corel.graph", "airs_default")]
res_corel_fit = run_cli(cmd_corel_fit, log_name="05_corel_fit")
print(res_corel_fit["stdout"][:300])

cmd_corel_eval = ["spectramind", "conformal", "corel-eval",
                  "--model", corel_dir, "--out", os.path.join(corel_dir, "eval.json")]
res_corel_eval = run_cli(cmd_corel_eval, log_name="06_corel_eval")
print(res_corel_eval["stdout"][:300])


## Diagnostics dashboard

In [ ]:
dash_path = os.path.join(DIAG_OUT, "diagnostic_report_uq_corel_v1.html")
cmd_dash = ["spectramind", "diagnose", "dashboard",
            "--out", dash_path,
            "--include-coverage", "true"]
res_dash = run_cli(cmd_dash, log_name="07_dashboard")
print(res_dash["stdout"][:300])
print("Dashboard:", dash_path)


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Train K ensemble members] --> B[Predict each + MC Dropout]
  B --> C[Aggregate μ,σ; compute epistemic variance]
  C --> D[Temp scaling (post-hoc)]
  D --> E[COREL conformal fit + eval (coverage)]
  E --> F[Diagnostics dashboard / CI]
```

## Next steps
- Sweep **ensemble size, MC samples, dropout p**, and **coverage** via `spectramind ablate`.
- Try **per-wavelength** temperature scaling (if supported) and compare GLL/coverage.
- Export JSON summaries for CI gating and archive HTML in versioned reports.
